In [ ]:
import psycopg2

# Connect to the remote database
conn = psycopg2.connect(
    host="hostname",
    database="database",
    user="username",
    password="password"
)

# Open the SQL export file
with open("sql_export.sql", "r") as file:
    sql = file.read()

# Create a cursor
cur = conn.cursor()

# Execute the SQL commands in the file
cur.execute(sql)

# Commit the changes to the database
conn.commit()

# Close the cursor and connection
cur.close()
conn.close()


In [ ]:
def get_db_connection():
    open_ssh_tunnel()
    
    return open_db_connection()


def open_db_connection() :
    try:
            self.conn = mysql.connector.MySQLConnection(
            host = '127.0.0.1',
            user = 'examwfgd_crawls',
            passwd = 'incorrect0727531915',
            db = 'examwfgd_crawls',
            port=self.tunnel.local_bind_port,
            use_pure=True,
            charset='utf8',
            use_unicode=True
        )
        except Exception as e:
            self.connection_attempts = 0
            self.logger.info(f"A connection error has occured")
            if self.connection_attempts <= 3:
                self.logger.info(f"Connection attempt: {self.connection_attempts}")

                self.connection_attempts = self.connection_attempts + 1

                self.open_ssh_tunnel()
                self.connect_to_mysql()
            else:
                self.logger.info(f"Unable to connect to db after multiple attempts")
        
            
        else:
            self.logger.info("Successfully connected to remote database")
            

def open_ssh_tunnel():
    if verbose:
        sshtunnel.DEFAULT_LOGLEVEL = logging.DEBUG

    ssh_password = "incorrect0727531915"


    security_key = paramiko.RSAKey.from_private_key_file((os.getcwd() +"\\keys\\database_key"),ssh_password) 
    #security_key = paramiko.RSAKey.from_private_key_file(self.connection_details.security_filepath, self.connection_details.ssh_password)

    sshtunnel.SSH_TIMEOUT = 6000.0
    sshtunnel.TUNNEL_TIMEOUT = 12000.0

    tunnel = sshtunnel.SSHTunnelForwarder(
            ('198.54.115.176', 21098),
            ssh_private_key=security_key,
            ssh_username = 'examwfgd',
            set_keepalive = 12000,
            remote_bind_address = ('127.0.0.1', 3306)
        )
        
    tunnel.start()

In [ ]:
import mysql.connector
import os

# Connect to the local database
local_conn = mysql.connector.connect(
    host="localhost",
    user="kush",
    password="incorrect0727531915",
    database="examwfgd_crawls"
)

# Connect to the remote database
remote_conn = get_db_connection()

# Get the table names from the local database
local_cursor = local_conn.cursor()
local_cursor.execute("SHOW TABLES")
tables = local_cursor.fetchall()

# Loop through each table
for table in tables:
    # Get the column names from the local database
    local_cursor.execute(f"DESCRIBE {table[0]}")
    columns = [column[0] for column in local_cursor.fetchall()]

    # Get the last exported record ID
    last_exported_id = 0

    # Number of records per chunk
    records_per_chunk = 10000

    while True:
        # Get the next chunk of records
        local_cursor.execute(f"SELECT * FROM {table[0]} WHERE id > {last_exported_id} LIMIT {records_per_chunk}")
        records = local_cursor.fetchall()

        # Break the loop if there are no more records
        if not records:
            break

        # Generate the SQL statement to insert the records
        sql = f"INSERT INTO {table[0]} ({', '.join(columns)}) VALUES "
        for record in records:
            values = [str(value) for value in record]
            sql += f"('{', '.join(values)}'),"

        # Remove the trailing comma
        sql = sql[:-1]

        # Execute the SQL statement
        remote_conn.cursor().execute(sql)

        # Commit the changes
        remote_conn.commit()

        # Update the last exported record ID
        last_exported_id = records[-1][0]

# Close the connections
local_conn.close()
remote_conn.close()
